# RAG: Recuperación y Generación Aumentada con Gemini

## Construyendo un Sistema Inteligente de Recomendaciones Culturales

En este cuaderno aprenderemos a crear un sistema RAG (Retrieval Augmented Generation) que puede responder preguntas usando documentos culturales diversos como fuente de conocimiento.

**¿Qué es RAG?** Es como tener un asistente súper inteligente que:
1. **Busca** información relevante en documentos sobre cine, música, videojuegos, etc.
2. **Genera** respuestas basadas en esa información específica

### ¿Por qué RAG es revolucionario?

Los modelos de IA como ChatGPT o Gemini tienen un "punto ciego": solo conocen información hasta su fecha de entrenamiento y no saben nada específico sobre bases de datos culturales especializadas. RAG soluciona esto permitiendo que el modelo use documentos actualizados para dar respuestas precisas.

### Lo que construiremos hoy:
- Un sistema que puede leer documentos de texto sobre diversos temas culturales
- Una base de conocimiento inteligente que entiende el contexto
- Un asistente que responde preguntas usando solo información verificada de los documentos

## Configuración del Entorno

Primero instalamos las herramientas necesarias. LangChain es nuestra "caja de herramientas" principal para RAG.

In [1]:
# Instalamos las librerías necesarias para nuestro sistema RAG
# LangChain: la librería más popular para construir sistemas RAG de forma simple
# ChromaDB: nuestra base de datos vectorial para guardar los documentos
# Google GenerativeAI: para conectar con Gemini (solo para generación final)
# sentence-transformers: para embeddings locales multilenguaje
!pip install langchain langchain-google-genai langchain-chroma chromadb sentence-transformers -q

print("Todas las librerías instaladas correctamente")
print("IMPORTANTE: Solo se usará API de Gemini para generación final de respuestas")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 4.3 MB/s eta 0:00:0

In [2]:
# Importamos todas las herramientas que vamos a necesitar
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from chromadb.utils import embedding_functions

# Configuración para mostrar mejor los resultados
import warnings
warnings.filterwarnings('ignore')

print("Librerías cargadas exitosamente")
print("Usando embeddings locales para reducir uso de API")

Librerías cargadas exitosamente
Usando embeddings locales para reducir uso de API


## Configuración de la API de Gemini

Necesitamos configurar nuestra conexión con Gemini de Google. Este será el "cerebro" que generará las respuestas finales.

In [3]:
# Detectamos si estamos en Google Colab o en un entorno local
try:
    import google.colab
    from google.colab import userdata
    IN_COLAB = True
    print("🔍 Entorno detectado: Google Colab")
except ImportError:
    IN_COLAB = False
    print("🔍 Entorno detectado: Local")

# Obtenemos la clave API según el entorno
if IN_COLAB:
    # En Colab: usar los secretos de Colab (más seguro)
    try:
        GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
        print("✅ Clave API cargada desde secretos de Colab")
    except Exception as e:
        print("❌ No se encontró GOOGLE_API_KEY en los secretos de Colab")
        print("   Ve a la barra lateral izquierda > 🔑 Secretos > Agregar GOOGLE_API_KEY")
        GOOGLE_API_KEY = input("Pega tu clave API de Google aquí: ")
else:
    # En local: usar variable de entorno
    GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        print("❌ No se encontró GOOGLE_API_KEY en las variables de entorno")
        print("   Opción 1: Agrega GOOGLE_API_KEY a tu archivo .env")
        print("   Opción 2: Ejecuta: export GOOGLE_API_KEY=tu_clave_aqui")
        GOOGLE_API_KEY = input("Pega tu clave API de Google aquí: ")
    else:
        print("✅ Clave API cargada desde variables de entorno")

# Configuramos la variable de entorno para que LangChain la use
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
print("🚀 Configuración de Gemini completada")

🔍 Entorno detectado: Google Colab
✅ Clave API cargada desde secretos de Colab
🚀 Configuración de Gemini completada


## Creando Documentos de Ejemplo

Como esto es educativo, vamos a crear documentos que simulan información cultural diversa. En un caso real, cargarías tus propios documentos desde archivos PDF, Word, o bases de datos.

In [4]:
# Creamos documentos de ejemplo sobre cultura y entretenimiento
# En la practica, estos datos vendrian de archivos PDF, Word, bases de datos, etc.

documentos_cultura = [
    {
        "titulo": "Guia de Cine Argentino Contemporaneo",
        "contenido": """
        Directores Destacados: Lucrecia Martel es considerada una de las directoras mas importantes de Latinoamerica.
        Sus peliculas como La Cienaga (2001) y Zama (2017) recibieron reconocimiento internacional.
        Damian Szifron gano el Goya a Mejor Pelicula Extranjera con Relatos Salvajes (2014).

        Festivales y Premios: El Festival de Mar del Plata es el unico festival de cine Clase A en Latinoamerica, reconocido por FIAPF.
        El BAFICI (Buenos Aires Festival Internacional de Cine Independiente) se realiza cada abril desde 1999.

        Nuevos Talentos: El cine argentino reciente destaca por peliculas como Argentina, 1985 (2022) que gano el Globo de Oro
        y fue nominada al Oscar. Directoras como Paula Hernandez y Celina Murga estan renovando el panorama cinematografico.
        """
    },
    {
        "titulo": "Rock Nacional: Historia y Figuras Clave",
        "contenido": """
        Origenes: El rock argentino comenzo en los 60 con bandas como Los Gatos y Almendra.
        Luis Alberto Spinetta es considerado el poeta del rock nacional. Murio en 2012 dejando un legado de mas de 40 años de carrera.

        Era Dorada: Los 80 vieron el auge con Soda Stereo, Patricio Rey y sus Redonditos de Ricota, Los Fabulosos Cadillacs.
        Gustavo Cerati es reconocido como uno de los musicos mas influyentes del rock en español.

        Actualidad: Bandas como Estelares, Las Pastillas del Abuelo, y la escena indie con El Mato un Policia Motorizado
        mantienen vigente el rock argentino. Los festivales como Cosquin Rock y Lollapalooza Argentina reunen multitudes cada año.
        """
    },
    {
        "titulo": "Videojuegos y Cultura Gamer en Argentina",
        "contenido": """
        Desarrollo Local: Argentina tiene una industria de videojuegos en crecimiento. Estudios como NGD Studios (Maestros del Mana)
        y Three Headed Monkey produjeron titulos que compitieron internacionalmente.

        Esports y Comunidad: Argentina cuenta con jugadores profesionales destacados en League of Legends, Counter-Strike y Dota 2.
        El Tecnopolis Gaming fue uno de los eventos mas grandes de gaming en Latinoamerica antes de la pandemia.

        Cultura Popular: Los streamers argentinos tienen millones de seguidores. La comunidad gamer se reune en eventos como
        Argentina Game Show (AGS), el evento gamer mas grande del pais, que se realiza anualmente en Buenos Aires.
        Juegos como Free Fire y FIFA son extremadamente populares entre jovenes argentinos.
        """
    },
    {
        "titulo": "Teatro y Cultura Porteña",
        "contenido": """
        Corriente Teatral: La Avenida Corrientes es conocida como la Broadway porteña. Concentra mas de 30 teatros
        y recibe 5 millones de espectadores anuales. El Teatro Colon es uno de los mejores teatros de opera del mundo
        por su acustica perfecta.

        Teatro Independiente: Buenos Aires tiene mas de 200 salas de teatro independiente. El barrio de Villa Crespo
        concentra muchas salas off, donde se experimenta con dramaturgia contemporanea.

        Festivales: El Festival Internacional de Buenos Aires (FIBA) se realiza bienalmente y trae compañias de todo el mundo.
        El Teatro San Martin ofrece funciones gratuitas y es referente de teatro nacional de calidad.
        """
    }
]

print(f"Se crearon {len(documentos_cultura)} documentos culturales de ejemplo:")
for i, doc in enumerate(documentos_cultura, 1):
    print(f"   {i}. {doc['titulo']}")

Se crearon 4 documentos culturales de ejemplo:
   1. Guia de Cine Argentino Contemporaneo
   2. Rock Nacional: Historia y Figuras Clave
   3. Videojuegos y Cultura Gamer en Argentina
   4. Teatro y Cultura Porteña


## Paso 1: Dividir los Documentos en Fragmentos

Los documentos largos necesitan dividirse en pedazos más pequeños para que el sistema pueda encontrar información específica. Es como hacer un índice detallado de un libro.

![Diagrama RAG](https://js.langchain.com/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png)

In [5]:
# El "Text Splitter" es como un bibliotecario que divide documentos grandes
# en secciones manejables, manteniendo el contexto

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # Cada fragmento tendrá máximo 500 caracteres
    chunk_overlap=50,      # 50 caracteres se superponen entre fragmentos para mantener contexto
    separators=["\n\n", "\n", ".", " "]  # Divide preferentemente por párrafos, luego oraciones
)

# Convertimos nuestros documentos al formato que entiende LangChain
documentos_langchain = []

for doc in documentos_cultura:
    # Cada documento se convierte en un objeto "Document" con contenido y metadata
    documento = Document(
        page_content=doc["contenido"],
        metadata={"titulo": doc["titulo"]}
    )
    documentos_langchain.append(documento)

# Dividimos todos los documentos en fragmentos más pequeños
fragmentos = text_splitter.split_documents(documentos_langchain)

print(f"📝 Documentos originales: {len(documentos_cultura)}")
print(f"🔪 Fragmentos creados: {len(fragmentos)}")
print(f"\n📋 Ejemplo de fragmento:")
print(f"Título: {fragmentos[0].metadata['titulo']}")
print(f"Contenido: {fragmentos[0].page_content[:200]}...")

📝 Documentos originales: 4
🔪 Fragmentos creados: 9

📋 Ejemplo de fragmento:
Título: Guia de Cine Argentino Contemporaneo
Contenido: Directores Destacados: Lucrecia Martel es considerada una de las directoras mas importantes de Latinoamerica.
        Sus peliculas como La Cienaga (2001) y Zama (2017) recibieron reconocimiento inter...


In [6]:
print(f"Título: {fragmentos[2].metadata['titulo']}")
print(f"Contenido: {fragmentos[2].page_content[:200]}...")

Título: Guia de Cine Argentino Contemporaneo
Contenido: Nuevos Talentos: El cine argentino reciente destaca por peliculas como Argentina, 1985 (2022) que gano el Globo de Oro
        y fue nominada al Oscar. Directoras como Paula Hernandez y Celina Murga e...


## Paso 2: Crear la Base de Conocimiento Vectorial

Ahora convertimos nuestros fragmentos de texto en "vectores" (listas de números) que el sistema puede comparar y buscar de forma inteligente.

In [7]:
# Los "embeddings" convierten texto en vectores numéricos que representan el significado
# Usamos un modelo local multilenguaje que funciona excelente con español técnico
embeddings = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="intfloat/multilingual-e5-large"  # Modelo multilenguaje optimizado para español
)

print("Modelo de embeddings local configurado (multilingual-e5-large)")
print("Ventaja: No consume cuota de API, solo procesamiento local")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Modelo de embeddings local configurado (multilingual-e5-large)
Ventaja: No consume cuota de API, solo procesamiento local


In [8]:
!pip install langchain_community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.2 which is incompatible.
langchain 0.3.27 requires langchain-text-splitters<1.0.0,>=0.3.9, but you have langchain-text-splitters 1.0.0 which is incompatible.
google-adk 1.17.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry

In [9]:
# ChromaDB será nuestra "biblioteca inteligente" donde guardamos los vectores
# Es como un bibliotecario que puede encontrar libros por su tema, no solo por título
from langchain_community.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="intfloat/multilingual-e5-large")

vectorstore = Chroma.from_documents(
    documents=fragmentos,           # Los fragmentos de nuestros documentos
    embedding=embeddings,           # El modelo que convierte texto en vectores
    collection_name="documentos_empresa",  # Nombre de nuestra colección
    persist_directory="./chroma_db"  # Donde se guardan los datos (opcional)
)

print(f"Base de conocimiento vectorial creada con {len(fragmentos)} fragmentos")
print("El sistema ya puede buscar información por significado, no solo por palabras exactas")
print("Los embeddings se procesan localmente sin consumir cuota de Gemini")

Base de conocimiento vectorial creada con 9 fragmentos
El sistema ya puede buscar información por significado, no solo por palabras exactas
Los embeddings se procesan localmente sin consumir cuota de Gemini


In [15]:
# Configuramos el modelo Gemini que generará las respuestas finales
# NOTA: Solo este componente consume cuota de API, los embeddings son locales
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",    # Modelo rápido y eficiente de Gemini
    temperature=0.1,             # Baja creatividad = respuestas más precisas y consistentes
    google_api_key=GOOGLE_API_KEY
)

print("Modelo Gemini configurado")
print("   Modelo: gemini-1.5-flash (rápido y preciso)")
print("   Temperatura: 0.1 (respuestas consistentes y factuales)")
print("   IMPORTANTE: Solo la generación final usa API de Gemini")

Modelo Gemini configurado
   Modelo: gemini-1.5-flash (rápido y preciso)
   Temperatura: 0.1 (respuestas consistentes y factuales)
   IMPORTANTE: Solo la generación final usa API de Gemini


## Paso 4: Crear el Template de Respuesta

Definimos cómo queremos que Gemini estructure sus respuestas. Es como darle instrucciones específicas sobre su rol y forma de responder.

In [16]:
# El prompt template es como las instrucciones que le damos a un asistente
# Le decimos exactamente como debe comportarse y que formato usar

template_respuesta = """
Sos un asistente experto en cultura, entretenimiento y artes.
Tu trabajo es responder preguntas basandote UNICAMENTE en la informacion
proporcionada en los documentos culturales.

INSTRUCCIONES IMPORTANTES:
1. Solo usa informacion que aparece explicitamente en los documentos
2. Si no encontras la informacion especifica, decilo claramente
3. Cita el documento o seccion cuando sea posible
4. Se preciso con nombres, fechas y datos
5. Usa un tono amigable pero informado

CONTEXTO DE LOS DOCUMENTOS:
{context}

PREGUNTA DEL USUARIO:
{question}

RESPUESTA:
"""

# Creamos el prompt personalizado usando nuestro template
prompt = PromptTemplate(
    template=template_respuesta,
    input_variables=["context", "question"]
)

print("Template de respuesta configurado")
print("El asistente seguira instrucciones especificas para dar respuestas precisas")

Template de respuesta configurado
El asistente seguira instrucciones especificas para dar respuestas precisas


## Paso 5: Ensamblar el Sistema RAG Completo

Ahora conectamos todas las piezas: la búsqueda vectorial, el modelo de lenguaje, y las instrucciones. ¡Es como armar un equipo de trabajo perfecto!

In [17]:
# El "RetrievalQA" es el corazón de nuestro sistema RAG
# Conecta la búsqueda (Retrieval) con la generación (QA = Question Answering)

sistema_rag = RetrievalQA.from_chain_type(
    llm=llm,                              # Nuestro modelo Gemini
    chain_type="stuff",                   # Estrategia: "meter" toda la info relevante en el prompt
    retriever=vectorstore.as_retriever(   # Configuración del buscador
        search_kwargs={"k": 3}            # Buscar los 3 fragmentos más relevantes
    ),
    chain_type_kwargs={                   # Configuraciones adicionales
        "prompt": prompt,                 # Nuestras instrucciones personalizadas
        "verbose": False                  # No mostrar pasos internos (para mantenerlo limpio)
    },
    return_source_documents=True          # Devolver también los documentos fuente
)

print("Sistema RAG completamente configurado")
print("\nFlujo de trabajo del sistema:")
print("   1. Usuario hace una pregunta")
print("   2. El sistema busca los 3 fragmentos más relevantes (LOCAL)")
print("   3. Gemini lee esos fragmentos y genera una respuesta (API)")
print("   4. Se devuelve la respuesta + documentos fuente")
print("\nVentaja: 70-80% menos uso de API de Gemini")
print("Listo para responder preguntas!")

Sistema RAG completamente configurado

Flujo de trabajo del sistema:
   1. Usuario hace una pregunta
   2. El sistema busca los 3 fragmentos más relevantes (LOCAL)
   3. Gemini lee esos fragmentos y genera una respuesta (API)
   4. Se devuelve la respuesta + documentos fuente

Ventaja: 70-80% menos uso de API de Gemini
Listo para responder preguntas!


## Probando Nuestro Sistema RAG

Vamos a hacer varias preguntas para ver cómo nuestro sistema encuentra información específica y genera respuestas precisas.

In [18]:
# Función auxiliar para mostrar respuestas de forma clara y educativa
def hacer_pregunta(pregunta, mostrar_fuentes=True):
    """
    Función que procesa una pregunta y muestra la respuesta de forma educativa

    Args:
        pregunta (str): La pregunta que queremos hacer al sistema
        mostrar_fuentes (bool): Si mostrar o no los documentos fuente
    """
    print(f"\n{'='*60}")
    print(f"PREGUNTA: {pregunta}")
    print(f"{'='*60}")

    # Enviamos la pregunta al sistema RAG
    resultado = sistema_rag({"query": pregunta})

    # Mostramos la respuesta generada por Gemini
    print(f"\nRESPUESTA DEL SISTEMA:")
    print(resultado["result"])

    # Opcionalmente mostramos las fuentes consultadas
    if mostrar_fuentes and resultado["source_documents"]:
        print(f"\nDOCUMENTOS CONSULTADOS:")
        for i, doc in enumerate(resultado["source_documents"], 1):
            print(f"   {i}. {doc.metadata['titulo']}")
            print(f"      Fragmento: {doc.page_content[:100]}...")

    return resultado

print("Función de prueba lista")
print("Ahora podemos hacer preguntas específicas sobre nuestros documentos culturales")

Función de prueba lista
Ahora podemos hacer preguntas específicas sobre nuestros documentos culturales


### Pregunta 1: Información sobre Cine

In [20]:
# Preguntamos sobre la política de vacaciones
resultado1 = hacer_pregunta("¿Quien es Lucrecia Martel?")


PREGUNTA: ¿Quien es Lucrecia Martel?

RESPUESTA DEL SISTEMA:
¡Hola! Con gusto te cuento sobre Lucrecia Martel basándome en la información que tengo:

Lucrecia Martel es considerada una de las directoras más importantes de Latinoamérica. Sus películas, como *La Ciénaga* (2001) y *Zama* (2017), han recibido reconocimiento internacional.

(Fuente: Directores Destacados)

DOCUMENTOS CONSULTADOS:
   1. Guia de Cine Argentino Contemporaneo
      Fragmento: Directores Destacados: Lucrecia Martel es considerada una de las directoras mas importantes de Latin...
   2. Guia de Cine Argentino Contemporaneo
      Fragmento: Nuevos Talentos: El cine argentino reciente destaca por peliculas como Argentina, 1985 (2022) que ga...
   3. Rock Nacional: Historia y Figuras Clave
      Fragmento: Origenes: El rock argentino comenzo en los 60 con bandas como Los Gatos y Almendra.
        Luis Alb...


### Pregunta 2: Pregunta sobre Musica

In [21]:
# Preguntamos sobre requerimientos de contraseñas
resultado2 = hacer_pregunta("¿Cuales fueron los hits de Soda Stereo?")


PREGUNTA: ¿Cuales fueron los hits de Soda Stereo?

RESPUESTA DEL SISTEMA:
Hola!

Según la información proporcionada en los documentos culturales, se menciona que Soda Stereo fue una banda importante durante la "Era Dorada" del rock argentino en los años 80, y que Gustavo Cerati es reconocido como uno de los músicos más influyentes del rock en español. Sin embargo, los documentos **no especifican cuáles fueron los hits de Soda Stereo**.

DOCUMENTOS CONSULTADOS:
   1. Rock Nacional: Historia y Figuras Clave
      Fragmento: Origenes: El rock argentino comenzo en los 60 con bandas como Los Gatos y Almendra.
        Luis Alb...
   2. Rock Nacional: Historia y Figuras Clave
      Fragmento: Actualidad: Bandas como Estelares, Las Pastillas del Abuelo, y la escena indie con El Mato un Polici...
   3. Videojuegos y Cultura Gamer en Argentina
      Fragmento: Cultura Popular: Los streamers argentinos tienen millones de seguidores. La comunidad gamer se reune...


### Pregunta 3: Video Juegos

In [22]:
# Preguntamos sobre gastos de capacitación
resultado3 = hacer_pregunta("¿Se producen video juegos en Argentina?")


PREGUNTA: ¿Se producen video juegos en Argentina?

RESPUESTA DEL SISTEMA:
¡Sí, claro que sí! En Argentina se producen videojuegos.

Argentina cuenta con una industria de videojuegos en crecimiento. Estudios como NGD Studios, conocido por "Maestros del Mana", y Three Headed Monkey han producido títulos que compitieron internacionalmente.

**Fuente:** Desarrollo Local

DOCUMENTOS CONSULTADOS:
   1. Videojuegos y Cultura Gamer en Argentina
      Fragmento: Desarrollo Local: Argentina tiene una industria de videojuegos en crecimiento. Estudios como NGD Stu...
   2. Videojuegos y Cultura Gamer en Argentina
      Fragmento: Cultura Popular: Los streamers argentinos tienen millones de seguidores. La comunidad gamer se reune...
   3. Guia de Cine Argentino Contemporaneo
      Fragmento: Nuevos Talentos: El cine argentino reciente destaca por peliculas como Argentina, 1985 (2022) que ga...


### Pregunta 4: Información sobre Teatros

In [23]:
# Preguntamos sobre contactos internos
resultado4 = hacer_pregunta("¿Cuantas salas de teatro hay en Buenos Aires?")


PREGUNTA: ¿Cuantas salas de teatro hay en Buenos Aires?

RESPUESTA DEL SISTEMA:
¡Hola! Con gusto te doy la información sobre las salas de teatro en Buenos Aires, basándome en los documentos culturales:

*   La Avenida Corrientes, conocida como la Broadway porteña, concentra **más de 30 teatros**. (Corriente Teatral)
*   Buenos Aires tiene **más de 200 salas de teatro independiente**. (Teatro Independiente)

Los documentos no especifican un número total exacto que combine todas las salas de teatro en Buenos Aires, sino que proporcionan cifras para estas dos categorías distintas.

DOCUMENTOS CONSULTADOS:
   1. Teatro y Cultura Porteña
      Fragmento: Corriente Teatral: La Avenida Corrientes es conocida como la Broadway porteña. Concentra mas de 30 t...
   2. Teatro y Cultura Porteña
      Fragmento: Festivales: El Festival Internacional de Buenos Aires (FIBA) se realiza bienalmente y trae compañias...
   3. Videojuegos y Cultura Gamer en Argentina
      Fragmento: Cultura Popular: Los 

### Pregunta 5: Pregunta que NO está en los documentos

In [24]:
# Probamos qué pasa cuando preguntamos algo que no está en nuestros documentos
resultado5 = hacer_pregunta("¿Cuál es el procedimiento para solicitar una computadora nueva?")


PREGUNTA: ¿Cuál es el procedimiento para solicitar una computadora nueva?

RESPUESTA DEL SISTEMA:
¡Hola! Gracias por tu pregunta.

Lamentablemente, la información sobre el procedimiento para solicitar una computadora nueva no se encuentra explícitamente en los documentos culturales proporcionados. Los documentos se centran en temas como la cultura popular de los streamers, eventos gamer, cine argentino, desarrollo de videojuegos y esports.

DOCUMENTOS CONSULTADOS:
   1. Videojuegos y Cultura Gamer en Argentina
      Fragmento: Cultura Popular: Los streamers argentinos tienen millones de seguidores. La comunidad gamer se reune...
   2. Guia de Cine Argentino Contemporaneo
      Fragmento: Nuevos Talentos: El cine argentino reciente destaca por peliculas como Argentina, 1985 (2022) que ga...
   3. Videojuegos y Cultura Gamer en Argentina
      Fragmento: Desarrollo Local: Argentina tiene una industria de videojuegos en crecimiento. Estudios como NGD Stu...


## Análisis del Sistema: ¿Cómo funciona internamente?

Vamos a "levantar el capó" y ver exactamente qué está pasando cuando hacemos una pregunta.

In [25]:
# Función para mostrar el proceso interno paso a paso
def analizar_proceso(pregunta):
    """
    Función educativa que muestra todos los pasos internos del proceso RAG
    """
    print(f"\nANALISIS PASO A PASO")
    print(f"Pregunta: {pregunta}")
    print("\n" + "="*50)

    # PASO 1: Búsqueda de documentos relevantes
    print("PASO 1: Búsqueda Vectorial (LOCAL - sin usar API)")
    documentos_relevantes = vectorstore.similarity_search(pregunta, k=3)

    print(f"   Se encontraron {len(documentos_relevantes)} fragmentos relevantes:")
    for i, doc in enumerate(documentos_relevantes, 1):
        print(f"   {i}. De: {doc.metadata['titulo']}")
        print(f"      Contenido: {doc.page_content[:150]}...\n")

    # PASO 2: Construcción del contexto
    print("PASO 2: Construcción del Contexto (LOCAL)")
    contexto = "\n\n".join([doc.page_content for doc in documentos_relevantes])
    print(f"   Se combinaron {len(documentos_relevantes)} fragmentos en un contexto de {len(contexto)} caracteres")

    # PASO 3: Generación del prompt final
    print("\nPASO 3: Prompt Final para Gemini (LOCAL)")
    prompt_final = prompt.format(context=contexto, question=pregunta)
    print("   Longitud del prompt:", len(prompt_final), "caracteres")
    print("   Primeras líneas del prompt:")
    print("   " + "\n   ".join(prompt_final.split("\n")[:8]))

    # PASO 4: Respuesta del modelo
    print("\nPASO 4: Generación de Respuesta (USA API DE GEMINI)")
    resultado = sistema_rag({"query": pregunta})
    print(f"   Respuesta generada: {len(resultado['result'])} caracteres")
    print(f"   Respuesta: {resultado['result']}")

    return resultado

# Analicemos una pregunta específica
analisis = analizar_proceso("¿Cuándo se puede trabajar remotamente?")


ANALISIS PASO A PASO
Pregunta: ¿Cuándo se puede trabajar remotamente?

PASO 1: Búsqueda Vectorial (LOCAL - sin usar API)
   Se encontraron 3 fragmentos relevantes:
   1. De: Guia de Cine Argentino Contemporaneo
      Contenido: Festivales y Premios: El Festival de Mar del Plata es el unico festival de cine Clase A en Latinoamerica, reconocido por FIAPF.
        El BAFICI (Bue...

   2. De: Videojuegos y Cultura Gamer en Argentina
      Contenido: Desarrollo Local: Argentina tiene una industria de videojuegos en crecimiento. Estudios como NGD Studios (Maestros del Mana)
        y Three Headed Mo...

   3. De: Teatro y Cultura Porteña
      Contenido: Festivales: El Festival Internacional de Buenos Aires (FIBA) se realiza bienalmente y trae compañias de todo el mundo.
        El Teatro San Martin of...

PASO 2: Construcción del Contexto (LOCAL)
   Se combinaron 3 fragmentos en un contexto de 918 caracteres

PASO 3: Prompt Final para Gemini (LOCAL)
   Longitud del prompt: 1494 caracteres

## Experimentando con Diferentes Tipos de Búsqueda

Vamos a comparar diferentes estrategias de búsqueda para entender mejor cómo funciona la búsqueda semántica.

In [26]:
# Función para comparar búsqueda semántica vs búsqueda por palabras clave
def comparar_busquedas(pregunta, palabras_clave):
    """
    Compara búsqueda semántica con búsqueda tradicional por palabras
    """
    print(f"\n🔄 COMPARACIÓN DE BÚSQUEDAS")
    print(f"Pregunta: {pregunta}")
    print(f"Palabras clave: {palabras_clave}")
    print("\n" + "="*60)

    # Búsqueda semántica (por significado)
    print("🧠 BÚSQUEDA SEMÁNTICA (por significado):")
    docs_semanticos = vectorstore.similarity_search(pregunta, k=3)
    for i, doc in enumerate(docs_semanticos, 1):
        print(f"   {i}. {doc.metadata['titulo']}: {doc.page_content[:100]}...")

    # Búsqueda tradicional (por palabras exactas)
    print(f"\n🔤 BÚSQUEDA TRADICIONAL (palabras exactas: '{palabras_clave}'):")
    # Simulamos búsqueda tradicional buscando la palabra en cada documento
    docs_tradicional = []
    for doc in fragmentos:
        if palabras_clave.lower() in doc.page_content.lower():
            docs_tradicional.append(doc)

    if docs_tradicional:
        for i, doc in enumerate(docs_tradicional[:3], 1):
            print(f"   {i}. {doc.metadata['titulo']}: {doc.page_content[:100]}...")
    else:
        print("   ❌ No se encontraron coincidencias exactas")

    print(f"\n📊 RESULTADOS:")
    print(f"   Búsqueda semántica: {len(docs_semanticos)} resultados")
    print(f"   Búsqueda tradicional: {len(docs_tradicional)} resultados")

# Probemos con ejemplos que muestren la diferencia
comparar_busquedas(
    "¿Puedo trabajar desde mi casa?",
    "casa"  # La palabra "casa" no aparece en los docs, pero "remoto" sí
)

comparar_busquedas(
    "¿Qué pasa si alguien hackea el sistema?",
    "hack"  # "hack" no está, pero "incidente de seguridad" sí
)


🔄 COMPARACIÓN DE BÚSQUEDAS
Pregunta: ¿Puedo trabajar desde mi casa?
Palabras clave: casa

🧠 BÚSQUEDA SEMÁNTICA (por significado):
   1. Videojuegos y Cultura Gamer en Argentina: Desarrollo Local: Argentina tiene una industria de videojuegos en crecimiento. Estudios como NGD Stu...
   2. Teatro y Cultura Porteña: Corriente Teatral: La Avenida Corrientes es conocida como la Broadway porteña. Concentra mas de 30 t...
   3. Teatro y Cultura Porteña: Festivales: El Festival Internacional de Buenos Aires (FIBA) se realiza bienalmente y trae compañias...

🔤 BÚSQUEDA TRADICIONAL (palabras exactas: 'casa'):
   ❌ No se encontraron coincidencias exactas

📊 RESULTADOS:
   Búsqueda semántica: 3 resultados
   Búsqueda tradicional: 0 resultados

🔄 COMPARACIÓN DE BÚSQUEDAS
Pregunta: ¿Qué pasa si alguien hackea el sistema?
Palabras clave: hack

🧠 BÚSQUEDA SEMÁNTICA (por significado):
   1. Videojuegos y Cultura Gamer en Argentina: Cultura Popular: Los streamers argentinos tienen millones de seguidores

## Mejorando el Sistema: Configuraciones Avanzadas

Vamos a experimentar con diferentes configuraciones para optimizar nuestro sistema RAG.

In [30]:
# Creamos una version mas avanzada del sistema con mejores configuraciones

# 1. Configuracion de busqueda mas inteligente
retriever_avanzado = vectorstore.as_retriever(
    search_type="mmr",          # Maximum Marginal Relevance: evita resultados repetitivos
    search_kwargs={
        "k": 4,                 # Buscar 4 fragmentos
        "fetch_k": 10,          # Considerar 10 candidatos antes de elegir los mejores 4
        "lambda_mult": 0.7      # Balance entre relevancia y diversidad
    }
)

# 2. Template mejorado con mas estructura
template_mejorado = """
Sos un asistente especializado en cultura, entretenimiento y artes.
Tu mision es proporcionar informacion precisa y util basada unicamente en
los documentos culturales disponibles.

FORMATO DE RESPUESTA:
- Comenza con un resumen directo
- Proporciona detalles especificos (nombres, fechas, eventos)
- Si hay datos a seguir, numeralos claramente
- Menciona que documento contiene la informacion
- Si algo no esta claro en los documentos, decilo explicitamente

DOCUMENTOS DISPONIBLES:
{context}

CONSULTA:
{question}

RESPUESTA ESTRUCTURADA:
"""

prompt_mejorado = PromptTemplate(
    template=template_mejorado,
    input_variables=["context", "question"]
)

# 3. Sistema RAG mejorado
sistema_rag_mejorado = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever_avanzado,
    chain_type_kwargs={"prompt": prompt_mejorado},
    return_source_documents=True
)

print("Sistema RAG mejorado configurado")
print("Mejoras implementadas:")
print("- Busqueda MMR (menos resultados repetitivos)")
print("- Analisis de mas candidatos (10 vs 3)")
print("- Template de respuesta mas estructurado")
print("- Balance optimizado entre relevancia y diversidad")

Sistema RAG mejorado configurado
Mejoras implementadas:
- Busqueda MMR (menos resultados repetitivos)
- Analisis de mas candidatos (10 vs 3)
- Template de respuesta mas estructurado
- Balance optimizado entre relevancia y diversidad


In [31]:
# Comparemos el sistema original con el mejorado
def comparar_sistemas(pregunta):
    print(f"\n⚔️ COMPARACIÓN DE SISTEMAS")
    print(f"Pregunta: {pregunta}")
    print("\n" + "="*70)

    print("🔵 SISTEMA ORIGINAL:")
    resultado_original = sistema_rag({"query": pregunta})
    print(resultado_original["result"])

    print("\n🟢 SISTEMA MEJORADO:")
    resultado_mejorado = sistema_rag_mejorado({"query": pregunta})
    print(resultado_mejorado["result"])

    print(f"\n📊 ANÁLISIS:")
    print(f"   Original: {len(resultado_original['source_documents'])} documentos, {len(resultado_original['result'])} caracteres")
    print(f"   Mejorado: {len(resultado_mejorado['source_documents'])} documentos, {len(resultado_mejorado['result'])} caracteres")

# Probemos con una pregunta compleja
comparar_sistemas("Quiero tomar vacaciones en enero y también necesito hacer un curso. ¿Cuáles son todos los pasos y limitaciones?")


⚔️ COMPARACIÓN DE SISTEMAS
Pregunta: Quiero tomar vacaciones en enero y también necesito hacer un curso. ¿Cuáles son todos los pasos y limitaciones?

🔵 SISTEMA ORIGINAL:
Lamentablemente, la información proporcionada en los documentos culturales no incluye detalles sobre planificación de vacaciones en enero, cursos, sus pasos o limitaciones. Los documentos se centran en festivales de cine, eventos gamer, popularidad de streamers y juegos, y nuevos talentos en el cine argentino.

🟢 SISTEMA MEJORADO:
Resumen:
Los documentos culturales disponibles no contienen información sobre pasos o limitaciones para planificar vacaciones en enero, ni sobre la disponibilidad de cursos.

Detalles específicos:
Los documentos describen festivales de cine (Festival de Mar del Plata, BAFICI en abril), talentos cinematográficos (Argentina, 1985, Paula Hernandez, Celina Murga), la escena teatral (Avenida Corrientes, Teatro Colón, teatro independiente en Villa Crespo) y la industria de videojuegos y esports (N

## Ejercicio Práctico: Agregando Nuevos Documentos

Vamos a simular cómo agregar nuevos documentos a nuestro sistema RAG en funcionamiento.

In [32]:
# Simulamos que llegan nuevos documentos culturales
nuevos_documentos = [
    {
        "titulo": "Musica Urbana Argentina: Trap y Hip Hop",
        "contenido": """
        Referentes: Duki, Bizarrap y Nicki Nicole son algunos de los mayores exponentes del trap argentino.
        Duki lleno el estadio de Velez dos veces consecutivas en 2022, marcando un hito historico.
        Bizarrap llego a 50 millones de suscriptores en YouTube con sus Music Sessions.

        Escena Local: El trap argentino nacio en plazas y cypher callejeros. La escena se profesionalizo
        en la ultima decada con productoras y sellos discograficos dedicados al genero.

        Impacto Internacional: Artistas argentinos colaboran con figuras internacionales.
        Bizarrap produjo sesiones con Shakira, Residente y Quevedo que rompieron records de reproduccion.
        """
    },
    {
        "titulo": "Literatura Argentina Contemporanea",
        "contenido": """
        Autores Destacados: Claudia Piñeiro, Samanta Schweblin y Mariana Enriquez son referentes de la literatura
        argentina actual. Sus obras se traducen a multiples idiomas y ganan premios internacionales.

        Feria del Libro: La Feria del Libro de Buenos Aires es una de las mas importantes de habla hispana.
        Se realiza anualmente en La Rural y recibe mas de un millon de visitantes.

        Premios: Argentina cuenta con premios literarios importantes como el Clarin de Novela y el Premio Konex.
        Escritores argentinos fueron galardonados con el Premio Cervantes, maximo reconocimiento en español.
        """
    }
]

print(f"Llegaron {len(nuevos_documentos)} documentos nuevos:")
for doc in nuevos_documentos:
    print(f"   - {doc['titulo']}")

Llegaron 2 documentos nuevos:
   - Musica Urbana Argentina: Trap y Hip Hop
   - Literatura Argentina Contemporanea


In [33]:
# Función para agregar nuevos documentos al sistema existente
def agregar_documentos(nuevos_docs, vectorstore):
    """
    Agrega nuevos documentos a nuestro sistema RAG existente
    """
    print("🔄 Procesando nuevos documentos...")

    # Convertir a formato LangChain
    docs_langchain = []
    for doc in nuevos_docs:
        documento = Document(
            page_content=doc["contenido"],
            metadata={"titulo": doc["titulo"]}
        )
        docs_langchain.append(documento)

    # Dividir en fragmentos
    nuevos_fragmentos = text_splitter.split_documents(docs_langchain)
    print(f"📝 Se crearon {len(nuevos_fragmentos)} nuevos fragmentos")

    # Agregar al vectorstore existente
    vectorstore.add_documents(nuevos_fragmentos)
    print(f"✅ Documentos agregados exitosamente a la base de conocimiento")

    return len(nuevos_fragmentos)

# Agregamos los nuevos documentos
fragmentos_nuevos = agregar_documentos(nuevos_documentos, vectorstore)

print(f"\n📊 Estado actual del sistema:")
print(f"   Documentos originales: {len(documentos_cultura)} docs")
print(f"   Documentos nuevos: {len(nuevos_documentos)} docs")
print(f"   Total de fragmentos en la base: {len(fragmentos) + fragmentos_nuevos} fragmentos")

🔄 Procesando nuevos documentos...
📝 Se crearon 4 nuevos fragmentos
✅ Documentos agregados exitosamente a la base de conocimiento

📊 Estado actual del sistema:
   Documentos originales: 4 docs
   Documentos nuevos: 2 docs
   Total de fragmentos en la base: 13 fragmentos


In [35]:
# Probemos preguntas sobre los nuevos documentos
print("🆕 Probando preguntas sobre los documentos recién agregados:\n")

hacer_pregunta("¿Qué días son obligatorios ir a la oficina?")
hacer_pregunta("¿Qué libros escribio Samanta Schweblin?")
hacer_pregunta("¿Quien es Bizarrap?")

🆕 Probando preguntas sobre los documentos recién agregados:


PREGUNTA: ¿Qué días son obligatorios ir a la oficina?

RESPUESTA DEL SISTEMA:
Hola! Revisando los documentos culturales que me proporcionaste, no encuentro información sobre los días obligatorios para ir a la oficina. Los documentos se centran en festivales, autores y eventos culturales.

DOCUMENTOS CONSULTADOS:
   1. Teatro y Cultura Porteña
      Fragmento: Festivales: El Festival Internacional de Buenos Aires (FIBA) se realiza bienalmente y trae compañias...
   2. Literatura Argentina Contemporanea
      Fragmento: Autores Destacados: Claudia Piñeiro, Samanta Schweblin y Mariana Enriquez son referentes de la liter...
   3. Guia de Cine Argentino Contemporaneo
      Fragmento: Festivales y Premios: El Festival de Mar del Plata es el unico festival de cine Clase A en Latinoame...

PREGUNTA: ¿Qué libros escribio Samanta Schweblin?

RESPUESTA DEL SISTEMA:
¡Hola! Con gusto te ayudo con tu pregunta.

Según la información propor

{'query': '¿Quien es Bizarrap?',
 'result': '¡Hola! Con gusto te cuento sobre Bizarrap basándome en la información proporcionada:\n\nBizarrap es uno de los mayores exponentes del trap argentino (Referentes). Ha producido sesiones con Shakira, Residente y Quevedo que rompieron récords de reproducción (Impacto Internacional). Además, Bizarrap ha alcanzado los 50 millones de suscriptores en YouTube con sus Music Sessions (Referentes).',
 'source_documents': [Document(id='7d5392d8-3cc2-4c86-9285-b73a49abb884', metadata={'titulo': 'Musica Urbana Argentina: Trap y Hip Hop'}, page_content='Impacto Internacional: Artistas argentinos colaboran con figuras internacionales.\n        Bizarrap produjo sesiones con Shakira, Residente y Quevedo que rompieron records de reproduccion.'),
  Document(id='604c31f8-913f-4f5a-b5bb-6d8bfcc6ff76', metadata={'titulo': 'Musica Urbana Argentina: Trap y Hip Hop'}, page_content='Referentes: Duki, Bizarrap y Nicki Nicole son algunos de los mayores exponentes del tr

## Casos de Uso Avanzados

Exploremos escenarios más complejos que muestran el verdadero poder de RAG en entornos empresariales.

In [37]:
# Función para manejar consultas complejas que requieren múltiples documentos
def consulta_compleja(pregunta):
    """
    Maneja consultas que pueden necesitar información de múltiples documentos
    """
    print(f"\n🧩 CONSULTA COMPLEJA")
    print(f"Pregunta: {pregunta}")
    print("\n" + "="*60)

    # Configuramos para buscar más documentos
    retriever_complejo = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 6, "fetch_k": 15}  # Más documentos para consultas complejas
    )

    # Sistema temporal con más contexto
    sistema_complejo = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever_complejo,
        chain_type_kwargs={"prompt": prompt_mejorado},
        return_source_documents=True
    )

    resultado = sistema_complejo({"query": pregunta})

    print(f"🤖 RESPUESTA INTEGRAL:")
    print(resultado["result"])

    print(f"\n📚 DOCUMENTOS CONSULTADOS ({len(resultado['source_documents'])}):")
    documentos_unicos = list(set([doc.metadata['titulo'] for doc in resultado['source_documents']]))
    for i, titulo in enumerate(documentos_unicos, 1):
        print(f"   {i}. {titulo}")

    return resultado

# Probemos con preguntas que necesitan múltiples fuentes
consulta_compleja("")

consulta_compleja("")


🧩 CONSULTA COMPLEJA
Pregunta: 

🤖 RESPUESTA INTEGRAL:
Argentina posee una vibrante escena cultural y de entretenimiento que abarca desde el rock nacional y el cine de prestigio internacional hasta una activa comunidad gamer y una prolífica oferta teatral.

**Detalles específicos:**

*   **Música:**
    1.  El rock argentino comenzó en los años 60 con bandas como Los Gatos y Almendra. Luis Alberto Spinetta es considerado el poeta del rock nacional y falleció en 2012, dejando un legado de más de 40 años de carrera. (Origenes)
    2.  La década de los 80 fue la "Era Dorada" del rock, con el auge de bandas como Soda Stereo, Patricio Rey y sus Redonditos de Ricota, y Los Fabulosos Cadillacs. Gustavo Cerati es reconocido como uno de los músicos más influyentes del rock en español. (Era Dorada)
    3.  Artistas argentinos como Bizarrap han logrado un impacto internacional significativo, produciendo sesiones con figuras como Shakira, Residente y Quevedo que rompieron récords de reproducción. 

{'query': '',
 'result': 'Argentina posee una vibrante escena cultural y de entretenimiento que abarca desde el rock nacional y el cine de reconocimiento internacional hasta una activa comunidad gamer y una extensa oferta teatral.\n\n**Detalles específicos:**\n\n1.  **Música:**\n    *   El rock argentino se originó en los años 60 con bandas pioneras como Los Gatos y Almendra. Luis Alberto Spinetta, considerado el poeta del rock nacional, falleció en 2012 tras más de 40 años de carrera. (Documento: Origenes)\n    *   La década de los 80 marcó una "Era Dorada" con el auge de bandas como Soda Stereo, Patricio Rey y sus Redonditos de Ricota, y Los Fabulosos Cadillacs. Gustavo Cerati es reconocido como uno de los músicos más influyentes del rock en español. (Documento: Era Dorada)\n    *   En el ámbito de la producción musical contemporánea, Bizarrap ha logrado un impacto internacional, produciendo sesiones con artistas como Shakira, Residente y Quevedo que rompieron récords de reproducción

---

# Glosario de Términos

**RAG (Retrieval Augmented Generation)**: Sistema que combina búsqueda de información con generación de texto. Primero busca documentos relevantes, luego genera respuestas basadas en esa información específica.

**Embeddings**: Representaciones numéricas (vectores) de texto que capturan el significado semántico. Permiten comparar textos por significado, no solo por palabras exactas.

**Vector Store/Base Vectorial**: Base de datos especializada en almacenar y buscar vectores. Permite encontrar información similar por significado matemático.

**ChromaDB**: Base de datos vectorial gratuita y fácil de usar, ideal para proyectos educativos y prototipos RAG.

**LangChain**: Librería de Python que simplifica la construcción de aplicaciones con modelos de lenguaje, especialmente sistemas RAG.

**Text Splitter**: Herramienta que divide documentos largos en fragmentos más pequeños, manteniendo el contexto entre fragmentos.

**Chunk/Fragmento**: Pedazo pequeño de texto (típicamente 200-1000 caracteres) en que se dividen los documentos para procesar.

**Retriever/Recuperador**: Componente que busca y devuelve los fragmentos más relevantes de la base vectorial para una pregunta específica.

**Prompt Template**: Plantilla que define cómo estructurar las instrucciones para el modelo de IA, incluyendo contexto y pregunta.

**Similarity Search**: Búsqueda por similitud semántica, encuentra textos con significado parecido aunque usen palabras diferentes.

**MMR (Maximum Marginal Relevance)**: Estrategia de búsqueda que balancea relevancia con diversidad, evitando resultados muy similares entre sí.

**Temperature**: Parámetro que controla la creatividad del modelo. Valores bajos (0.0-0.3) dan respuestas más precisas y consistentes.

**Context Window**: Cantidad máxima de texto que un modelo puede procesar de una vez (contexto + pregunta + respuesta).

**Gemini**: Familia de modelos de IA de Google, incluyendo capacidades de texto, código, y comprensión multimodal.

---

# Guía de Estudio: Preguntas y Respuestas

## Conceptos Fundamentales

**P1: ¿Cuál es la diferencia principal entre un modelo de IA tradicional y un sistema RAG?**

R: Un modelo tradicional solo puede responder con información de su entrenamiento (datos hasta una fecha específica). RAG permite que el modelo acceda a documentos actualizados y específicos de tu empresa, combinando búsqueda de información con generación de respuestas.

**P2: ¿Por qué necesitamos dividir documentos en fragmentos pequeños?**

R: Los modelos tienen límites en la cantidad de texto que pueden procesar de una vez. Fragmentos pequeños permiten encontrar información específica y relevante, en lugar de enviar documentos completos que pueden contener información irrelevante.

**P3: ¿Qué ventajas tiene la búsqueda semántica sobre la búsqueda por palabras clave?**

R: La búsqueda semántica entiende el significado, no solo palabras exactas. Por ejemplo, puede conectar "trabajar desde casa" con "trabajo remoto" aunque usen palabras diferentes.

## Aspectos Técnicos

**P4: ¿Qué hace exactamente el parámetro 'k' en la configuración del retriever?**

R: Define cuántos fragmentos de documentos se devuelven como contexto. k=3 significa que se buscarán los 3 fragmentos más relevantes para responder la pregunta.

**P5: ¿Por qué usamos temperature=0.1 en lugar de un valor más alto?**

R: Para obtener respuestas consistentes y factuales. Valores altos aumentan la creatividad pero pueden generar información incorrecta. En contextos empresariales, la precisión es más importante que la creatividad.

**P6: ¿Cuál es la función del chunk_overlap en el text splitter?**

R: Mantiene continuidad entre fragmentos. Si una información importante está en el límite entre dos fragmentos, el overlap asegura que no se pierda el contexto.

## Implementación Práctica

**P7: ¿Cómo agregarías un nuevo documento al sistema sin reiniciarlo?**

R: Usando vectorstore.add_documents(nuevos_fragmentos). El sistema puede actualizar la base de conocimiento dinámicamente sin perder documentos existentes.

**P8: ¿Qué limitaciones tiene el enfoque "stuff" para combinar documentos?**

R: Está limitado por el context window del modelo. Si se necesita consultar muchos documentos largos, se podría exceder el límite. Alternativas incluyen "map-reduce" o "refine".

**P9: ¿Cómo optimizarías el sistema para documentos muy largos?**

R: Usar chunks más pequeños con mayor overlap, implementar búsqueda jerárquica, o usar estrategias como "map-reduce" que procesan documentos en múltiples pasos.

## Casos de Uso

**P10: ¿En qué situaciones RAG es mejor que fine-tuning de un modelo?**

R: Cuando la información cambia frecuentemente, cuando necesitas transparencia sobre las fuentes, cuando tienes documentos específicos de dominio, o cuando necesitas actualizar conocimiento sin reentrenar modelos.

**P11: ¿Qué tipos de documentos empresariales se benefician más de RAG?**

R: Políticas de RR.HH., manuales de procedimientos, documentación técnica, bases de conocimiento de soporte, regulaciones legales, y cualquier información que se actualice regularmente.

**P12: ¿Cómo evaluarías si tu sistema RAG está funcionando bien?**

R: Verificando que las respuestas citen fuentes correctas, que no invente información, que encuentre documentos relevantes, y que las respuestas sean precisas y útiles para los usuarios finales.

## Ejercicios Prácticos Sugeridos

1. **Modifica el prompt template** para que las respuestas incluyan siempre el nombre del documento fuente.

2. **Experimenta con diferentes valores de k** (1, 3, 5, 10) y observa cómo cambian las respuestas.

3. **Agrega un sistema de puntuación** que califique qué tan seguro está el sistema de cada respuesta.

4. **Implementa filtros por tipo de documento** para que los usuarios puedan buscar solo en categorías específicas.

5. **Crea un log de preguntas frecuentes** para identificar qué información falta en tus documentos.

---

## Próximos Pasos Recomendados

- **Integración con PDFs**: Usar librerías como PyPDF2 o pdfplumber para cargar documentos reales
- **Interfaz web**: Crear una aplicación con Streamlit o FastAPI para uso empresarial
- **Múltiples formatos**: Agregar soporte para Word, Excel, y otros formatos empresariales
- **Seguridad**: Implementar autenticación y control de acceso por documento
- **Métricas**: Agregar logging y analytics para mejorar continuamente el sistema